In [2]:
import pandas as pd

In [3]:
df = pd.read_excel('A_Site_기상_Data(1분 주기)_20190128.xlsx', header=1)

In [4]:
df.to_pickle('C:/Users/ewpadmin/JIHYUN/A_site_real_climate.pkl')

In [106]:
df.head(3)

,관측시간,기온(°C),일누적강수량(mm),풍향(deg),풍속(m/s),상대습도(%),현지기압(hPa),해면기압(hPa),일누적일사(MJ/m^2),일누적일조(Sec)
0,07-01 00:00:00,22.6,0.0,168.3,0.0,98.1,1005.5,1008.8,14.32,20460.0
1,07-01 00:01:00,22.6,0.0,0.0,0.0,98.1,1005.5,1008.8,0.00,0.0
2,07-01 00:02:00,22.6,0.0,0.0,0.0,98.1,1005.5,1008.8,0.00,0.0


### NaN index reset

In [4]:
df = df.reset_index(drop=True)

In [5]:
df.head(3)

,관측시간,기온(°C),일누적강수량(mm),풍향(deg),풍속(m/s),상대습도(%),현지기압(hPa),해면기압(hPa),일누적일사(MJ/m^2),일누적일조(Sec)
0,07-01 00:00:00,22.6,0.0,168.3,0.0,98.1,1005.5,1008.8,14.32,20460.0
1,07-01 00:01:00,22.6,0.0,0.0,0.0,98.1,1005.5,1008.8,0.00,0.0
2,07-01 00:02:00,22.6,0.0,0.0,0.0,98.1,1005.5,1008.8,0.00,0.0


In [6]:
len(df)

201221

In [7]:
df.dtypes

관측시간              object
기온(°C)           float64
일누적강수량(mm)       float64
풍향(deg)          float64
풍속(m/s)          float64
상대습도(%)          float64
현지기압(hPa)        float64
해면기압(hPa)        float64
일누적일사(MJ/m^2)    float64
일누적일조(Sec)       float64
dtype: object

### [결측치] (1/29 토의)
- 1)데이터가 순차적일 때 : Rolling or 앞,뒤 값, 평균 등
- 2)강수량 : 0
- 3)이상현상을 보이는 데이터들
  - 결측치로 보이는 자리에 0이 들어가 있음
  - 누적데이터인데 감소하는 값이 들어가 있음
    - 해결방법: 함수를 사용해 순간 결측치로 컬럼을 변경하고 음수가 나오는 부분만 모아 따로 처리
### (2/7 토의)
- 기온 15분 평균
- 강수량 0
- 풍속, 상대습도, 기압, 해면기압, 15분 평균
- 풍향 8단계 360/8 45단위  
    1) 0은 그대로 0 (풍속이 0일 때 풍향도 0)  
    2) NaN은 바로 전 row의 값 (15분 평균을 하면 360도 전후로 바뀌는 부분에서 error 발생)  
    3) 360도를 45도 단위로 끊어서 예보와 같이 8방위로 
    
    |:---||:---||:---|
    |:북풍||:337.5||:22.5|  
    
- 일누적일사: (아래 순서로 진행)  
    1)find NaN -> fillna(0)  
    2)find 갑자기 감소하는 구간(Reset 구간):  
        if [i < i-1] & [일누적일사[i] != 0] -> 다시 누적으로 데이터를 이어 줌  
        날짜가 바뀌어서 리셋되는 것은 빠지도록 따로 처리해 줌    
    3)find 0  (결측치 구간):  
        find 0 & if [i-1 !=0]:  
            이전 15분 치의 linear regression을 따른 값(대개는 등차수열)  

In [150]:
df.isna().sum()
print(df.columns)
df[df.isnull().any(axis=1)].head(3)

Index(['관측시간', '기온(°C)', '일누적강수량(mm)', '풍향(deg)', '풍속(m/s)', '상대습도(%)',
       '현지기압(hPa)', '해면기압(hPa)', '일누적일사(MJ/m^2)', '일누적일조(Sec)'],
      dtype='object')


,관측시간,기온(°C),일누적강수량(mm),풍향(deg),풍속(m/s),상대습도(%),현지기압(hPa),해면기압(hPa),일누적일사(MJ/m^2),일누적일조(Sec)
11503,07-08 23:58:00,24.6,74.9,NaN,NaN,98.1,1005.0,1008.3,6.14,1920.0
11504,07-08 23:59:00,24.6,74.9,NaN,NaN,98.1,1005.0,1008.3,6.14,1920.0
11505,07-09 00:00:00,24.6,74.9,NaN,NaN,98.1,1004.9,1008.2,6.14,1920.0


In [350]:
df_temp = df.copy()

In [351]:
df_temp.isna().sum()

관측시간                 0
기온(°C)             154
일누적강수량(mm)       10465
풍향(deg)            482
풍속(m/s)            455
상대습도(%)            102
현지기압(hPa)          178
해면기압(hPa)          178
일누적일사(MJ/m^2)       58
일누적일조(Sec)          46
dtype: int64

In [168]:
# 두가지 방법 결과 같음 결측치 인덱스 확인
# df_temp.loc[df_temp['기온(°C)'].isnull(), '기온(°C)'].index
# df_temp[df_temp['기온(°C)'].isnull()].index

In [352]:
def min_15_mean(X, col):
    miss_idx = []
    miss_idx = X[X[col].isnull()].index

    for i in miss_idx:
        X[col][i] = round(X[col][i-15:i].sum()/ 15, 1)
    
    return df_temp

In [353]:
df_temp.columns[4:8]

Index(['풍속(m/s)', '상대습도(%)', '현지기압(hPa)', '해면기압(hPa)'], dtype='object')

### 기온, 풍속, 상대습도, 기압, 해면기압, 15분 평균

In [354]:
min_15_mean(df_temp, df_temp.columns[1])
print("기온")
min_15_mean(df_temp, df_temp.columns[4])
print("풍속")
min_15_mean(df_temp, df_temp.columns[5])
print("상대습도")
min_15_mean(df_temp, df_temp.columns[6])
print("현지기압")
min_15_mean(df_temp, df_temp.columns[7])
print("해면기압")

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


기온
풍속
상대습도
현지기압
해면기압


In [355]:
df_temp.isna().sum()

관측시간                 0
기온(°C)               0
일누적강수량(mm)       10465
풍향(deg)            482
풍속(m/s)              0
상대습도(%)              0
현지기압(hPa)            0
해면기압(hPa)            0
일누적일사(MJ/m^2)       58
일누적일조(Sec)          46
dtype: int64

In [356]:
# for i in miss_idx:
#     df_temp[df_temp.columns[1]][i] = round(df_temp[df_temp.columns[1]][i-15:i].sum()/ 15, 1)

#     print(df_temp[df_temp.columns[1]][i])
#     print(i)

# round(df_temp[df_temp.columns[1]][60561-15:60561].sum()/ 15, 1)
# df_temp[df_temp.columns[1]][60561]
# df_temp[df_temp.columns[1]][miss_idx].head(3)
# df_temp.isna().sum()

### 풍향

In [357]:
missing_idx = df_temp[df_temp['풍향(deg)'].isnull()].index

In [358]:
for i in missing_idx:
    df_temp['풍향(deg)'][i] = df_temp['풍향(deg)'][i-1]

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [359]:
df_temp.isnull().sum()

관측시간                 0
기온(°C)               0
일누적강수량(mm)       10465
풍향(deg)              0
풍속(m/s)              0
상대습도(%)              0
현지기압(hPa)            0
해면기압(hPa)            0
일누적일사(MJ/m^2)       58
일누적일조(Sec)          46
dtype: int64

In [360]:
def wind_direction(degree):
    if degree >= 337.5 or degree < 22.5:
        return '북풍'
    elif 22.5 <= degree < 67.5:
        return '북동풍'
    elif 67.5 <= degree < 112.5:
        return '동풍'
    elif 112.5 <= degree < 157.5:
        return '남동풍'
    elif 157.5 <= degree < 202.5:
        return '남풍'
    elif 202.5 <= degree < 247.5:
        return '남서풍'
    elif 247.5 <= degree < 292.5:
        return '서풍'
    else:
        return '북서풍'

In [361]:
df_temp['풍향(deg)'] = df_temp['풍향(deg)'].apply(lambda x: wind_direction(x))

In [362]:
df_temp.isnull().sum()

관측시간                 0
기온(°C)               0
일누적강수량(mm)       10465
풍향(deg)              0
풍속(m/s)              0
상대습도(%)              0
현지기압(hPa)            0
해면기압(hPa)            0
일누적일사(MJ/m^2)       58
일누적일조(Sec)          46
dtype: int64

In [364]:
df_temp.head()

,관측시간,기온(°C),일누적강수량(mm),풍향(deg),풍속(m/s),상대습도(%),현지기압(hPa),해면기압(hPa),일누적일사(MJ/m^2),일누적일조(Sec)
0,07-01 00:00:00,22.6,0.0,남풍,0.0,98.1,1005.5,1008.8,14.32,20460.0
1,07-01 00:01:00,22.6,0.0,북풍,0.0,98.1,1005.5,1008.8,0.00,0.0
2,07-01 00:02:00,22.6,0.0,북풍,0.0,98.1,1005.5,1008.8,0.00,0.0
3,07-01 00:03:00,22.6,0.0,북풍,0.0,98.1,1005.5,1008.8,0.00,0.0
4,07-01 00:04:00,22.5,0.0,북풍,0.0,98.1,1005.5,1008.8,0.00,0.0


### 강수량

In [365]:
df_temp.columns[2]

'일누적강수량(mm)'

In [366]:
df_temp.isna().sum()

관측시간                 0
기온(°C)               0
일누적강수량(mm)       10465
풍향(deg)              0
풍속(m/s)              0
상대습도(%)              0
현지기압(hPa)            0
해면기압(hPa)            0
일누적일사(MJ/m^2)       58
일누적일조(Sec)          46
dtype: int64

In [367]:
df_temp[df_temp.columns[2]] = df_temp[df_temp.columns[2]].fillna(0)

In [368]:
df_temp.isna().sum()

관측시간              0
기온(°C)            0
일누적강수량(mm)        0
풍향(deg)           0
풍속(m/s)           0
상대습도(%)           0
현지기압(hPa)         0
해면기압(hPa)         0
일누적일사(MJ/m^2)    58
일누적일조(Sec)       46
dtype: int64

### 일누적일사

In [369]:
df_temp.columns[8]

'일누적일사(MJ/m^2)'

In [370]:
df_temp[df_temp.columns[8]] = df_temp[df_temp.columns[8]].fillna(0)

In [371]:
df_temp.isna().sum()

관측시간              0
기온(°C)            0
일누적강수량(mm)        0
풍향(deg)           0
풍속(m/s)           0
상대습도(%)           0
현지기압(hPa)         0
해면기압(hPa)         0
일누적일사(MJ/m^2)     0
일누적일조(Sec)       46
dtype: int64

In [393]:
#이상한놈 찾기
for i in range(len(df_temp)-1):
    if (df_temp[df_temp.columns[8]][i] > df_temp[df_temp.columns[8]][i+1]) and df_temp[df_temp.columns[8]][i+1] != 0:
        print(i)
        print(df_temp[df_temp.columns[[0,8]]][i-1:i+10])
        print("-"*50)

65795
                   관측시간  일누적일사(MJ/m^2)
65794   08-15  20:09:00           3.42
65795   08-15  20:10:00           3.42
65796   08-15  20:11:00           3.41
65797   08-15  20:12:00           3.42
65798   08-15  20:13:00           3.42
65799   08-15  20:14:00           3.42
65800   08-15  20:15:00           3.42
65801   08-15  20:16:00           3.42
65802   08-15  20:17:00           3.42
65803   08-15  20:18:00           3.42
65804   08-15  20:19:00           3.42
--------------------------------------------------
70818
                   관측시간  일누적일사(MJ/m^2)
70817   08-19  07:53:00           0.28
70818   08-19  07:54:00           4.60
70819   08-19  07:55:00           0.29
70820   08-19  07:56:00           0.30
70821   08-19  07:57:00           0.30
70822   08-19  07:58:00           0.31
70823   08-19  07:59:00           0.31
70824   08-19  08:00:00           0.31
70825   08-19  08:01:00           0.32
70826   08-19  08:02:00           0.32
70827   08-19  08:03:00           0.33
-

In [373]:
df_temp[df_temp.columns[[0,8]]][65795-2:65795+5]

,관측시간,일누적일사(MJ/m^2)
65793,08-15 20:08:00,3.42
65794,08-15 20:09:00,3.42
65795,08-15 20:10:00,3.42
65796,08-15 20:11:00,3.41
65797,08-15 20:12:00,3.42
65798,08-15 20:13:00,3.42
65799,08-15 20:14:00,3.42


In [374]:
df_temp[df_temp.columns[[0,8]]][70818-2:70818+5]

,관측시간,일누적일사(MJ/m^2)
70816,08-19 07:52:00,0.28
70817,08-19 07:53:00,0.28
70818,08-19 07:54:00,4.60
70819,08-19 07:55:00,0.29
70820,08-19 07:56:00,0.30
70821,08-19 07:57:00,0.30
70822,08-19 07:58:00,0.31


In [375]:
df_temp[df_temp.columns[[0,8]]][70829-2:70829+5]

,관측시간,일누적일사(MJ/m^2)
70827,08-19 08:03:00,0.33
70828,08-19 08:04:00,0.33
70829,08-19 08:05:00,5.39
70830,08-19 08:06:00,0.34
70831,08-19 08:07:00,0.35
70832,08-19 08:08:00,0.35
70833,08-19 08:09:00,0.36


In [376]:
df_temp[df_temp.columns[[0,8]]][75112-2:75112+5]

,관측시간,일누적일사(MJ/m^2)
75110,08-22 07:46:00,0.31
75111,08-22 07:47:00,0.31
75112,08-22 07:48:00,20.20
75113,08-22 07:49:00,0.32
75114,08-22 07:50:00,0.32
75115,08-22 07:51:00,0.33
75116,08-22 07:52:00,0.33


### find 갑자기 감소하는 구간(Reset 구간): 
- if [i < i-1] & [일누적일사[i] != 0] -> 다시 누적으로 데이터를 이어 줌  
- 날짜가 바뀌어서 리셋되는 것은 빠지도록 따로 처리해 줌 

In [377]:
###이상한 놈들
df_temp[df_temp.columns[8]][65796] = df_temp[df_temp.columns[8]][65795]
df_temp[df_temp.columns[8]][70818] = df_temp[df_temp.columns[8]][70817]
df_temp[df_temp.columns[8]][70829] = df_temp[df_temp.columns[8]][70828]
df_temp[df_temp.columns[8]][75112] = df_temp[df_temp.columns[8]][75111]

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCo

In [394]:
for i in range(len(df_temp)-1):
    if (df_temp[df_temp.columns[8]][i] > df_temp[df_temp.columns[8]][i+1]) and df_temp[df_temp.columns[8]][i+1] != 0:
        df_temp[df_temp.columns[8]][i+1] = df_temp[df_temp.columns[8]][i]

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [395]:
#이상한놈 찾기
for i in range(len(df_temp)-1):
    if (df_temp[df_temp.columns[8]][i] > df_temp[df_temp.columns[8]][i+1]) and df_temp[df_temp.columns[8]][i+1] != 0:
        print(i)
        print(df_temp[df_temp.columns[[0,8]]][i-1:i+10])
        print("-"*50)

### find 0 (결측치 구간): 
- find 0 & if [i-1 !=0]:  
- 이전 15분 치의 linear regression을 따른 값(대개는 등차수열)

In [399]:
#이상한놈 찾기
for i in range(len(df_temp)-1):
    if "00:01:00" not in df_temp[df_temp.columns[0]][i+1] and \
    (df_temp[df_temp.columns[8]][i] > df_temp[df_temp.columns[8]][i+1]):
        print(df_temp[df_temp.columns[[0,8]]][i:i+10])

In [397]:
df_temp[df_temp.columns[[0,8]]][76498:76498+10]

,관측시간,일누적일사(MJ/m^2)
76498,08-23 06:55:00,7.64
76499,08-23 06:56:00,0.00
76500,08-23 06:57:00,0.00
76501,08-23 06:58:00,0.01
76502,08-23 06:59:00,0.01
76503,08-23 07:00:00,0.01
76504,08-23 07:01:00,0.02
76505,08-23 07:02:00,0.02
76506,08-23 07:03:00,0.03
76507,08-23 07:04:00,0.03


In [398]:
for i in range(len(df_temp)-1):
    if "00:01:00" not in df_temp[df_temp.columns[0]][i+1] and \
    (df_temp[df_temp.columns[8]][i] > df_temp[df_temp.columns[8]][i+1]):
        if i >= 76498:
            df_temp[df_temp.columns[8]][i+1] = df_temp[df_temp.columns[8]][76498] + df_temp[df_temp.columns[8]][i+1]
        else :
            df_temp[df_temp.columns[8]][i+1] = df_temp[df_temp.columns[8]][i] + df_temp[df_temp.columns[8]][i+1]  

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [ ]:
for i in range(len(df_temp)-1):
    if "00:01:00" not in df_temp[df_temp.columns[0]][i+1] and \
    (df_temp[df_temp.columns[8]][i] > df_temp[df_temp.columns[8]][i+1]):
        df_temp[df_temp.columns[[0,8]]][i:i+10]

In [400]:
df_temp[df_temp.columns[[0,8]]][76498:76498+10]

,관측시간,일누적일사(MJ/m^2)
76498,08-23 06:55:00,7.64
76499,08-23 06:56:00,7.64
76500,08-23 06:57:00,7.64
76501,08-23 06:58:00,7.65
76502,08-23 06:59:00,7.65
76503,08-23 07:00:00,7.65
76504,08-23 07:01:00,7.66
76505,08-23 07:02:00,7.66
76506,08-23 07:03:00,7.67
76507,08-23 07:04:00,7.67


### LinearRegression 으로 해보기

In [412]:
df_temp = df.copy()

In [413]:
df_temp[df_temp.columns[8]][65796] = df_temp[df_temp.columns[8]][65795]
df_temp[df_temp.columns[8]][70818] = df_temp[df_temp.columns[8]][70817]
df_temp[df_temp.columns[8]][70829] = df_temp[df_temp.columns[8]][70828]
df_temp[df_temp.columns[8]][75112] = df_temp[df_temp.columns[8]][75111]

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: Set

In [414]:
for i in range(len(df_temp)-1):
    if (df_temp[df_temp.columns[8]][i] > df_temp[df_temp.columns[8]][i+1]) and df_temp[df_temp.columns[8]][i+1] != 0:
        df_temp[df_temp.columns[8]][i+1] = df_temp[df_temp.columns[8]][i]

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [416]:
#이상한놈 찾기
for i in range(len(df_temp)-1):
    if "00:01:00" not in df_temp[df_temp.columns[0]][i+1] and \
    (df_temp[df_temp.columns[8]][i] > df_temp[df_temp.columns[8]][i+1]):
        print(df_temp[df_temp.columns[[0,8]]][i:i+10])

                   관측시간  일누적일사(MJ/m^2)
60712   08-12  07:23:00           0.33
60713   08-12  07:24:00           0.00
60714   08-12  07:25:00           0.34
60715   08-12  07:26:00           0.35
60716   08-12  07:27:00           0.36
60717   08-12  07:28:00           0.37
60718   08-12  07:29:00           0.38
60719   08-12  07:30:00           0.39
60720   08-12  07:31:00           0.40
60721   08-12  07:32:00           0.41
                   관측시간  일누적일사(MJ/m^2)
60728   08-12  07:39:00           0.49
60729   08-12  07:40:00           0.00
60730   08-12  07:41:00           0.52
60731   08-12  07:42:00           0.52
60732   08-12  07:43:00           0.54
60733   08-12  07:44:00           0.55
60734   08-12  07:45:00           0.56
60735   08-12  07:46:00           0.58
60736   08-12  07:47:00           0.58
60737   08-12  07:48:00           0.60
                   관측시간  일누적일사(MJ/m^2)
60744   08-12  07:55:00           0.70
60745   08-12  07:56:00           0.00
60746   08-12  07:57:00  

In [417]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

In [426]:
df_temp[100-5:100].index.shape, df_temp[df_temp.columns[8]][100-5:100].shape

((5,), (5,))

In [440]:
np.array(df_temp[df_temp.columns[8]][100-5:100])

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [468]:
np.array(df_temp[100-5:100].index).reshape(-1,1)

array([[95],
       [96],
       [97],
       [98],
       [99]], dtype=int64)

In [467]:
np.array(df_temp[df_temp.columns[8]][100-5:100]).reshape(-1,1)

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [494]:
np.array(df_temp[100:100+1].index).reshape(-1,1)

array([[100]], dtype=int64)

In [511]:
for i in range(len(df_temp)-1):
    if "00:01:00" not in df_temp[df_temp.columns[0]][i+1] and \
    (df_temp[df_temp.columns[8]][i] > df_temp[df_temp.columns[8]][i+1]):
        model = model.fit(np.array(df_temp[i-5:i].index).reshape(-1,1), \
                          np.array(df_temp[df_temp.columns[8]][i-5:i]).reshape(-1,1))
        y_new = model.predict(np.array(df_temp[i:i+1].index).reshape(-1,1))
        print(y_new)
        print(y_new.reshape(1,))
        print(y_new[0])
        df_temp[df_temp.columns[8]][i+1] = y_new[0]

[[0.33]]
[0.33]
[0.33]
[[0.494]]
[0.494]
[0.494]
[[0.695]]
[0.695]
[0.695]


C:\Anaconda\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


[[3.601]]
[3.601]
[3.601]
[[4.155]]
[4.155]
[4.155]
[[4.5428]]
[4.5428]
[4.5428]
[[4.2935]]
[4.2935]
[4.2935]
[[7.63]]
[7.63]
[7.63]
[[7.63]]
[7.63]
[7.63]
[[7.63]]
[7.63]
[7.63]
[[7.638]]
[7.638]
[7.638]
[[7.643]]
[7.643]
[7.643]
[[7.6434]]
[7.6434]
[7.6434]
[[7.6454]]
[7.6454]
[7.6454]
[[7.64382]]
[7.64382]
[7.64382]
[[7.64682]]
[7.64682]
[7.64682]
[[7.646936]]
[7.646936]
[7.646936]
[[7.646906]]
[7.646906]
[7.646906]
[[7.6478228]]
[7.6478228]
[7.6478228]
[[7.6478148]]
[7.6478148]
[7.6478148]
[[7.64888844]]
[7.64888844]
[7.64888844]
[[7.64812284]]
[7.64812284]
[7.64812284]
[[7.64911771]]
[7.64911771]
[7.64911771]
[[7.64896077]]
[7.64896077]
[7.64896077]
[[7.64922268]]
[7.64922268]
[7.64922268]
[[7.64933728]]
[7.64933728]
[7.64933728]
[[7.64931441]]
[7.64931441]
[7.64931441]
[[7.64971241]]
[7.64971241]
[7.64971241]
[[7.64942154]]
[7.64942154]
[7.64942154]
[[7.64978801]]
[7.64978801]
[7.64978801]
[[7.64963352]]
[7.64963352]
[7.64963352]
[[7.64981731]]
[7.64981731]
[7.64981731]
[[7.64978

[[7.65]]
[7.65]
[7.65]
[[7.65]]
[7.65]
[7.65]
[[7.65]]
[7.65]
[7.65]
[[7.65]]
[7.65]
[7.65]
[[7.65]]
[7.65]
[7.65]
[[7.65]]
[7.65]
[7.65]
[[7.65]]
[7.65]
[7.65]
[[7.65]]
[7.65]
[7.65]
[[7.65]]
[7.65]
[7.65]
[[7.65]]
[7.65]
[7.65]
[[7.65]]
[7.65]
[7.65]
[[7.65]]
[7.65]
[7.65]
[[7.65]]
[7.65]
[7.65]
[[7.65]]
[7.65]
[7.65]
[[7.65]]
[7.65]
[7.65]
[[7.65]]
[7.65]
[7.65]
[[7.65]]
[7.65]
[7.65]
[[7.65]]
[7.65]
[7.65]
[[7.65]]
[7.65]
[7.65]
[[7.65]]
[7.65]
[7.65]
[[7.65]]
[7.65]
[7.65]
[[7.65]]
[7.65]
[7.65]
[[7.65]]
[7.65]
[7.65]
[[7.65]]
[7.65]
[7.65]
[[7.65]]
[7.65]
[7.65]
[[7.65]]
[7.65]
[7.65]
[[7.65]]
[7.65]
[7.65]
[[7.65]]
[7.65]
[7.65]
[[7.65]]
[7.65]
[7.65]
[[7.65]]
[7.65]
[7.65]
[[7.65]]
[7.65]
[7.65]
[[7.65]]
[7.65]
[7.65]
[[7.65]]
[7.65]
[7.65]
[[7.65]]
[7.65]
[7.65]
[[7.65]]
[7.65]
[7.65]
[[7.65]]
[7.65]
[7.65]
[[7.65]]
[7.65]
[7.65]
[[7.65]]
[7.65]
[7.65]
[[7.65]]
[7.65]
[7.65]
[[7.65]]
[7.65]
[7.65]
[[7.65]]
[7.65]
[7.65]
[[7.65]]
[7.65]
[7.65]
[[7.65]]
[7.65]
[7.65]
[[7.65]]
[7

In [512]:
#이상한놈 찾기
for i in range(len(df_temp)-1):
    if "00:01:00" not in df_temp[df_temp.columns[0]][i+1] and \
    (df_temp[df_temp.columns[8]][i] > df_temp[df_temp.columns[8]][i+1]):
        print(df_temp[df_temp.columns[[0,8]]][i:i+10])

                   관측시간  일누적일사(MJ/m^2)
60712   08-12  07:23:00           0.33
60713   08-12  07:24:00           0.33
60714   08-12  07:25:00           0.34
60715   08-12  07:26:00           0.35
60716   08-12  07:27:00           0.36
60717   08-12  07:28:00           0.37
60718   08-12  07:29:00           0.38
60719   08-12  07:30:00           0.39
60720   08-12  07:31:00           0.40
60721   08-12  07:32:00           0.41
                   관측시간  일누적일사(MJ/m^2)
60744   08-12  07:55:00          0.700
60745   08-12  07:56:00          0.695
60746   08-12  07:57:00          0.730
60747   08-12  07:58:00          0.740
60748   08-12  07:59:00          0.750
60749   08-12  08:00:00          0.770
60750   08-12  08:01:00          0.780
60751   08-12  08:02:00          0.800
60752   08-12  08:03:00          0.810
60753   08-12  08:04:00          0.830
                   관측시간  일누적일사(MJ/m^2)
76454   08-23  06:10:00         4.5428
76455   08-23  06:11:00         4.2935
76456   08-23  06:12:00  

76548   08-23  07:45:00       7.650002
                   관측시간  일누적일사(MJ/m^2)
76541   08-23  07:38:00       7.650002
76542   08-23  07:39:00       7.649989
76543   08-23  07:40:00       7.649999
76544   08-23  07:41:00       7.649995
76545   08-23  07:42:00       7.649996
76546   08-23  07:43:00       7.650000
76547   08-23  07:44:00       7.649995
76548   08-23  07:45:00       7.650002
76549   08-23  07:46:00       7.649996
76550   08-23  07:47:00       7.650001
                   관측시간  일누적일사(MJ/m^2)
76543   08-23  07:40:00       7.649999
76544   08-23  07:41:00       7.649995
76545   08-23  07:42:00       7.649996
76546   08-23  07:43:00       7.650000
76547   08-23  07:44:00       7.649995
76548   08-23  07:45:00       7.650002
76549   08-23  07:46:00       7.649996
76550   08-23  07:47:00       7.650001
76551   08-23  07:48:00       7.649998
76552   08-23  07:49:00       7.649999
                   관측시간  일누적일사(MJ/m^2)
76546   08-23  07:43:00       7.650000
76547   08-23  07:44:00  

                   관측시간  일누적일사(MJ/m^2)
76624   08-23  09:01:00           7.65
76625   08-23  09:02:00           7.65
76626   08-23  09:03:00           7.65
76627   08-23  09:04:00           7.65
76628   08-23  09:05:00           7.65
76629   08-23  09:06:00           7.65
76630   08-23  09:07:00           7.65
76631   08-23  09:08:00           7.65
76632   08-23  09:09:00           7.65
76633   08-23  09:10:00           7.65
                   관측시간  일누적일사(MJ/m^2)
76626   08-23  09:03:00           7.65
76627   08-23  09:04:00           7.65
76628   08-23  09:05:00           7.65
76629   08-23  09:06:00           7.65
76630   08-23  09:07:00           7.65
76631   08-23  09:08:00           7.65
76632   08-23  09:09:00           7.65
76633   08-23  09:10:00           7.65
76634   08-23  09:11:00           7.65
76635   08-23  09:12:00           7.65
                   관측시간  일누적일사(MJ/m^2)
76628   08-23  09:05:00           7.65
76629   08-23  09:06:00           7.65
76630   08-23  09:07:00  

76685   08-23  10:02:00           7.65
                   관측시간  일누적일사(MJ/m^2)
76677   08-23  09:54:00           7.65
76678   08-23  09:55:00           7.65
76679   08-23  09:56:00           7.65
76680   08-23  09:57:00           7.65
76681   08-23  09:58:00           7.65
76682   08-23  09:59:00           7.65
76683   08-23  10:00:00           7.65
76684   08-23  10:01:00           7.65
76685   08-23  10:02:00           7.65
76686   08-23  10:03:00           7.65
                   관측시간  일누적일사(MJ/m^2)
76679   08-23  09:56:00           7.65
76680   08-23  09:57:00           7.65
76681   08-23  09:58:00           7.65
76682   08-23  09:59:00           7.65
76683   08-23  10:00:00           7.65
76684   08-23  10:01:00           7.65
76685   08-23  10:02:00           7.65
76686   08-23  10:03:00           7.65
76687   08-23  10:04:00           7.65
76688   08-23  10:05:00           7.65
                   관측시간  일누적일사(MJ/m^2)
76681   08-23  09:58:00           7.65
76682   08-23  09:59:00  

                   관측시간  일누적일사(MJ/m^2)
76752   08-23  11:09:00           7.65
76753   08-23  11:10:00           7.65
76754   08-23  11:11:00           7.65
76755   08-23  11:12:00           7.65
76756   08-23  11:13:00           7.65
76757   08-23  11:14:00           7.65
76758   08-23  11:15:00           7.65
76759   08-23  11:16:00           7.65
76760   08-23  11:17:00           7.65
76761   08-23  11:18:00           7.65
                   관측시간  일누적일사(MJ/m^2)
76756   08-23  11:13:00           7.65
76757   08-23  11:14:00           7.65
76758   08-23  11:15:00           7.65
76759   08-23  11:16:00           7.65
76760   08-23  11:17:00           7.65
76761   08-23  11:18:00           7.65
76762   08-23  11:19:00           7.65
76763   08-23  11:20:00           7.65
76764   08-23  11:21:00           7.65
76765   08-23  11:22:00           7.65
                   관측시간  일누적일사(MJ/m^2)
76758   08-23  11:15:00           7.65
76759   08-23  11:16:00           7.65
76760   08-23  11:17:00  

KeyboardInterrupt: 

### 일누적일조

In [160]:
df_temp.columns[9]

'일누적일조(Sec)'

In [162]:
df_temp = df_temp.drop(df_temp.columns[9], axis=1)
df_temp.head(3)

,관측시간,기온(°C),일누적강수량(mm),풍향(deg),풍속(m/s),상대습도(%),현지기압(hPa),해면기압(hPa),일누적일사(MJ/m^2)
0,07-01 00:00:00,22.6,0.0,168.3,0.0,98.1,1005.5,1008.8,14.32
1,07-01 00:01:00,22.6,0.0,0.0,0.0,98.1,1005.5,1008.8,0.00
2,07-01 00:02:00,22.6,0.0,0.0,0.0,98.1,1005.5,1008.8,0.00


### 15분 단위 변경

In [117]:
import numpy as np

def seq2dataset(X, col):
    Xdata = []
    for i in range(len(X) - 15):
        print(df.iloc[15*i:15+(15*i), col])
        print('-'*50)
        X_subset = df.iloc[15*i:15+(15*i), col]
        Xdata.append(X_subset)
        if i == 5:
            break
    
    return Xdata

In [118]:
df_15min = seq2dataset(df,1)

NaN    22.6
NaN    22.6
NaN    22.6
NaN    22.6
NaN    22.5
NaN    22.5
NaN    22.5
NaN    22.5
NaN    22.4
NaN    22.4
NaN    22.4
NaN    22.4
NaN    22.5
NaN    22.4
NaN    22.4
Name: 기온(°C), dtype: float64
--------------------------------------------------
NaN    22.5
NaN    22.4
NaN    22.3
NaN    22.3
NaN    22.3
NaN    22.3
NaN    22.3
NaN    22.3
NaN    22.2
NaN    22.2
NaN    22.2
NaN    22.2
NaN    22.3
NaN    22.3
NaN    22.3
Name: 기온(°C), dtype: float64
--------------------------------------------------
NaN    22.3
NaN    22.3
NaN    22.2
NaN    22.2
NaN    22.3
NaN    22.3
NaN    22.3
NaN    22.3
NaN    22.3
NaN    22.3
NaN    22.3
NaN    22.3
NaN    22.3
NaN    22.3
NaN    22.3
Name: 기온(°C), dtype: float64
--------------------------------------------------
NaN    22.3
NaN    22.3
NaN    22.3
NaN    22.3
NaN    22.3
NaN    22.4
NaN    22.4
NaN    22.4
NaN    22.4
NaN    22.5
NaN    22.5
NaN    22.4
NaN    22.4
NaN    22.4
NaN    22.4
Name: 기온(°C), dtype: float64
-----------

In [120]:
df_15min

[NaN    22.6
 NaN    22.6
 NaN    22.6
 NaN    22.6
 NaN    22.5
 NaN    22.5
 NaN    22.5
 NaN    22.5
 NaN    22.4
 NaN    22.4
 NaN    22.4
 NaN    22.4
 NaN    22.5
 NaN    22.4
 NaN    22.4
 Name: 기온(°C), dtype: float64, NaN    22.5
 NaN    22.4
 NaN    22.3
 NaN    22.3
 NaN    22.3
 NaN    22.3
 NaN    22.3
 NaN    22.3
 NaN    22.2
 NaN    22.2
 NaN    22.2
 NaN    22.2
 NaN    22.3
 NaN    22.3
 NaN    22.3
 Name: 기온(°C), dtype: float64, NaN    22.3
 NaN    22.3
 NaN    22.2
 NaN    22.2
 NaN    22.3
 NaN    22.3
 NaN    22.3
 NaN    22.3
 NaN    22.3
 NaN    22.3
 NaN    22.3
 NaN    22.3
 NaN    22.3
 NaN    22.3
 NaN    22.3
 Name: 기온(°C), dtype: float64, NaN    22.3
 NaN    22.3
 NaN    22.3
 NaN    22.3
 NaN    22.3
 NaN    22.4
 NaN    22.4
 NaN    22.4
 NaN    22.4
 NaN    22.5
 NaN    22.5
 NaN    22.4
 NaN    22.4
 NaN    22.4
 NaN    22.4
 Name: 기온(°C), dtype: float64, NaN    22.4
 NaN    22.4
 NaN    22.4
 NaN    22.4
 NaN    22.4
 NaN    22.3
 NaN    22.3
 NaN    2

In [ ]:
df.set_index('관측시간', inplace=True)

In [16]:
df = df.drop(df.columns[0], axis=1)
df.head()

,기온(°C),일누적강수량(mm),풍향(deg),풍속(m/s),상대습도(%),현지기압(hPa),해면기압(hPa),일누적일사(MJ/m^2),일누적일조(Sec)
NaN,22.6,0.0,168.3,0.0,98.1,1005.5,1008.8,14.32,20460.0
NaN,22.6,0.0,0.0,0.0,98.1,1005.5,1008.8,0.00,0.0
NaN,22.6,0.0,0.0,0.0,98.1,1005.5,1008.8,0.00,0.0
NaN,22.6,0.0,0.0,0.0,98.1,1005.5,1008.8,0.00,0.0
NaN,22.5,0.0,0.0,0.0,98.1,1005.5,1008.8,0.00,0.0
